In [1]:
library("dplyr")
library("ggplot2")
library("tidyr")


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [11]:
setwd("/workspace/hraijc/Blueberry/Blueberry_trio/contigbin/")

In [12]:
M7_ids <- read.csv(file = "./LinkageMap/M7_contig_ids.csv", header=T)
Nui_ids <- read.csv(file = "./LinkageMap/Nui_contig_ids.csv")

In [13]:
head(M7_ids)

,contig_result,reference_position,reference_contig,contig_allele,hx
,<chr>,<int>,<chr>,<chr>,<chr>
1,h2tg000457l_1,1068804,seq-0-001,T,h4
2,h1tg000606l_1,1068804,seq-0-001,C,ref
3,h1tg000606l_1,1215280,seq-0-001,C,ref
4,h2tg000457l_1,2717632,seq-0-001,C,ref
5,h1tg000423l_1,2717632,seq-0-001,C,ref
6,h1tg000423l_1,2176934,seq-0-001,T,ref


In [15]:
# Group by contig id and count the alleles adding a 0 if no data. 
M7_scores <- M7_ids %>% group_by(contig_result) %>% count(hx) %>% spread(key = hx, value = n,fill = 0 )

In [16]:
# subset removing ref column
M7_Sub <- select(M7_scores,-ref)

#Add row row_max printing rowwith max. 
M7_Sub <- M7_Sub %>% rowwise() %>% mutate(row_max = names(.)[which.max(c_across())+1])

# Add this row to original dataframe. 
M7_scores <- bind_cols(M7_scores, M7_Sub$row_max)

#Rename row
M7_scores$...10 -> M7_scores$hap

New names:
• `` -> `...10`


In [17]:
#Then add a column summing the number of SNPs for that contig. 
M7_scores <- M7_scores %>% mutate(SNPS_called=h1+h2+h3+h4+ref)

In [24]:
# Add column of the number of snps from the max hap.
M7_scores <- M7_scores %>%
  rowwise() %>%
  mutate(ALT = get(hap))

In [26]:
#Add column for the SNP_score
M7_scores <- M7_scores %>% mutate(SNP_score = ALT/SNPS_called)

In [31]:
# Set cutoff and make a new column only showing contig calls that are above that score:
CUTOFF <- 0.2
M7_scores <- M7_scores %>%
  rowwise() %>%
  mutate(hqcall = ifelse(SNP_score > CUTOFF, hap, "0"))

In [32]:
head(M7_scores)

contig_result,ERROR,h1,h2,h3,h4,insertion,nomap,ref,...10,hap,SNPS_called,ALT,SNP_score,hqcall
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
h1tg000002l_1,0,0,0,0,1,0,0,4,h4,h4,5,1,0.2000000,0
h1tg000004l_1,0,0,7,0,0,0,0,36,h2,h2,43,7,0.1627907,0
h1tg000005l_1,0,0,0,0,0,0,0,2,ERROR,ERROR,2,0,0.0000000,0
h1tg000006l_1,0,0,0,0,0,0,0,4,ERROR,ERROR,4,0,0.0000000,0
h1tg000007l_1,0,6,6,0,2,0,2,45,h1,h1,59,6,0.1016949,0
h1tg000008l_1,0,2,0,0,0,0,0,0,h1,h1,2,2,1.0000000,h1


In [33]:
head(M7_scores,n = 20)

contig_result,ERROR,h1,h2,h3,h4,insertion,nomap,ref,...10,hap,SNPS_called,ALT,SNP_score,hqcall
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
h1tg000002l_1,0,0,0,0,1,0,0,4,h4,h4,5,1,0.2000000,0
h1tg000004l_1,0,0,7,0,0,0,0,36,h2,h2,43,7,0.1627907,0
h1tg000005l_1,0,0,0,0,0,0,0,2,ERROR,ERROR,2,0,0.0000000,0
h1tg000006l_1,0,0,0,0,0,0,0,4,ERROR,ERROR,4,0,0.0000000,0
h1tg000007l_1,0,6,6,0,2,0,2,45,h1,h1,59,6,0.1016949,0
h1tg000008l_1,0,2,0,0,0,0,0,0,h1,h1,2,2,1.0000000,h1
h1tg000010l_1,0,0,0,0,0,0,0,2,ERROR,ERROR,2,0,0.0000000,0
h1tg000011l_1,0,0,0,0,0,0,0,3,ERROR,ERROR,3,0,0.0000000,0
h1tg000013l_1,0,1,0,0,0,0,0,3,h1,h1,4,1,0.2500000,h1


In [35]:
count(M7_scores,hqcall)

hqcall,n
<chr>,<int>
0,278
h1,66
h2,49
h3,62
h4,37
nomap,3
